In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pyspark.sql import functions as F  #filtering
import pandas as pd
import geopandas as gpd

In [2]:
# starting a Spark session
spark = (
    SparkSession.builder.appName('CriminalFirstCleaning')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

# read the parquet dataset
criminal = spark.read.parquet('../data/landing/criminal_incidents/criminal_table_03.parquet')

your 131072x1 screen size is bogus. expect trouble
24/09/07 20:42:50 WARN Utils: Your hostname, LAPTOP-KOQUIUN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/07 20:42:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/07 20:42:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/07 20:42:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Get the number of rows and columns
row_count = criminal.count()
column_count = len(criminal.columns)

# Return the shape of the cleaned DataFrame
criminal_shape = (row_count, column_count)
criminal_shape

(333900, 9)

In [4]:
# Get the number of duplicate rows directly
duplicate_count = criminal.count() - criminal.dropDuplicates().count()
duplicate_count

0

In [5]:
# Get the number of NA values for each column
na_counts = criminal.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in criminal.columns])
na_counts.show()

+----+-----------+---------------------+--------+----------------+----------------+-------------------+----------------+------------------+
|Year|Year ending|Local Government Area|Postcode|Suburb/Town Name|Offence Division|Offence Subdivision|Offence Subgroup|Incidents Recorded|
+----+-----------+---------------------+--------+----------------+----------------+-------------------+----------------+------------------+
|   0|          0|                    0|       0|               0|               0|                  0|               0|                 0|
+----+-----------+---------------------+--------+----------------+----------------+-------------------+----------------+------------------+



As we can see, there are no duplicates and NULL values on the dataset.

In [6]:
criminal.limit(5)

Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
2024,March,Alpine,3691,Dederang,B Property and de...,B40 Theft,B49 Other theft,1
2024,March,Alpine,3691,Dederang,D Public order an...,D10 Weapons and e...,D11 Firearms offe...,1
2024,March,Alpine,3691,Dederang,D Public order an...,D10 Weapons and e...,D13 Explosives of...,1
2024,March,Alpine,3691,Kergunyah South,A Crimes against ...,Other crimes agai...,Other crimes agai...,1
2024,March,Alpine,3691,Mongans Bridge,B Property and de...,B40 Theft,B49 Other theft,1


### Grouping

In [7]:
# Group by 'Offence Division', 'Offence Subdivision', and 'Offence Subgroup' and show unique groups
offence_groups = criminal.select("Offence Division", "Offence Subdivision", "Offence Subgroup").distinct()

# Show the grouped results
offence_groups.show(truncate=False)

+------------------------------------+---------------------------------------------------+------------------------------------------------------------------+
|Offence Division                    |Offence Subdivision                                |Offence Subgroup                                                  |
+------------------------------------+---------------------------------------------------+------------------------------------------------------------------+
|A Crimes against the person         |A20 Assault and related offences                   |A232 Non-FV Common assault                                        |
|D Public order and security offences|D30 Public nuisance offences                       |D35 Improper movement on public or private space                  |
|F Other offences                    |F30 Other government regulatory offences           |F39 Other government regulatory offences                          |
|F Other offences                    |F10 Regulatory

In [8]:
# Count the occurrences of each 'Offence Division'
offence_division_counts = criminal.groupBy("Offence Division").count().orderBy("Offence Division", ascending=True)

# Convert to Pandas for easier viewing (optional, if the dataset is small enough to fit in memory)
offence_division_summary = offence_division_counts.toPandas()

# Adjust the index to start from 1
offence_division_summary.index = offence_division_summary.index + 1

# Rename columns for better readability
offence_division_summary.columns = ['Offence Division', 'Count']

offence_division_summary

,Offence Division,Count
1,A Crimes against the person,93348
2,B Property and deception offences,133888
3,C Drug offences,22272
4,D Public order and security offences,38027
5,E Justice procedures offences,39488
6,F Other offences,6877


In [9]:
# Count the occurrences of each 'Offence Subdivision' and extract the first letter and number for sorting
offence_subdivision_counts = (
    criminal.groupBy("Offence Subdivision")
    .count()
    .withColumn("letter", F.substring(F.col("Offence Subdivision"), 1, 1))  # Extract first letter
    .withColumn("number", F.expr("cast(substring(`Offence Subdivision`, 2, length(`Offence Subdivision`)-1) as int)"))  # Extract number
    .orderBy("letter", "number")  # Order by both letter and number
    .drop("letter", "number")  # Drop the temporary sorting columns
)

# Convert to Pandas for easier viewing (optional)
offence_subdivision_summary = offence_subdivision_counts.toPandas()

# Adjust the index to start from 1
offence_subdivision_summary.index = offence_subdivision_summary.index + 1

# Rename columns for better readability
offence_subdivision_summary.columns = ['Offence Subdivision', 'Count']

offence_subdivision_summary

,Offence Subdivision,Count
1,A20 Assault and related offences,39669
2,A50 Robbery,5154
3,"A70 Stalking, harassment and threatening behav...",24088
4,A80 Dangerous and negligent acts endangering p...,13911
5,B20 Property damage,19995
6,B50 Deception,17389
7,B40 Theft,56086
8,B60 Bribery,47
9,B10 Arson,9622
10,B30 Burglary/Break and enter,30749


In [10]:
# Now do the same for 'Offence Subgroup'
offence_subgroup_counts = (
    criminal.groupBy("Offence Subgroup")
    .count()
    .withColumn("letter", F.substring(F.col("Offence Subgroup"), 1, 1))  # Extract first letter
    .withColumn("number", F.expr("cast(substring(`Offence Subgroup`, 2, length(`Offence Subgroup`)-1) as int)"))  # Extract number
    .orderBy("letter", "number")  # Order by both letter and number
    .drop("letter", "number")  # Drop the temporary sorting columns
)

# Convert to Pandas for easier viewing (optional)
offence_subgroup_summary = offence_subgroup_counts.toPandas()

# Adjust the index to start from 1
offence_subgroup_summary.index = offence_subgroup_summary.index + 1

# Rename columns for better readability
offence_subgroup_summary.columns = ['Offence Subgroup', 'Count']

offence_subgroup_summary

,Offence Subgroup,Count
1,A732 Non-FV Threatening behaviour,5148
2,A712 Non-FV Stalking,3584
3,A81 Dangerous driving,3819
4,A211 FV Serious assault,9075
5,A731 FV Threatening behaviour,5585
...,...,...
103,F36 Prostitution offences,127
104,F39 Other government regulatory offences,344
105,F14 Parking offences,3
106,F92 Public health and safety offences,2237


## Shape File

In [11]:
# sf stands for shape file
sf = gpd.read_file("../data/landing/boundaries/Victoria/vic_dist_boundaries.shp")

sf.head()

,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry
0,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78281 -37.56667, 143.75557 -37.5..."
1,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.81896 -37.55583, 143.81644 -37.5..."
2,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.8417 -37.61597, 143.84175 -37.61..."
3,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.75049 -37.5912, 143.75044 -37.59..."
4,201011007,Smythes Creek,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,104.7274,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.73295 -37.62334, 143.73262 -37.6..."


### Merge

In [12]:
# Convert the PySpark DataFrame to Pandas
criminal_pandas = criminal.toPandas()

# Perform the left join on 'suburb' from property and 'sa2_name' from sf
merged_df = criminal_pandas.merge(sf[['sa2_name', 'geometry']], 
                                  left_on='Suburb/Town Name', right_on='sa2_name', 
                                  how='left')

# Convert the merged DataFrame back to a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')

# Display the merged GeoDataFrame
merged_gdf.head(20)

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,sa2_name,geometry
0,2024,March,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN,None
1,2024,March,Alpine,3691,Dederang,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,1,NaN,None
2,2024,March,Alpine,3691,Dederang,D Public order and security offences,D10 Weapons and explosives offences,D13 Explosives offences,1,NaN,None
3,2024,March,Alpine,3691,Kergunyah South,A Crimes against the person,Other crimes against the person,Other crimes against the person,1,NaN,None
4,2024,March,Alpine,3691,Mongans Bridge,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN,None
5,2024,March,Alpine,3691,Running Creek,B Property and deception offences,B20 Property damage,B21 Criminal damage,2,NaN,None
6,2024,March,Alpine,3697,Tawonga,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1,NaN,None
7,2024,March,Alpine,3697,Tawonga,B Property and deception offences,B40 Theft,B49 Other theft,1,NaN,None
8,2024,March,Alpine,3698,Tawonga South,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1,NaN,None
9,2024,March,Alpine,3698,Tawonga South,B Property and deception offences,B20 Property damage,B21 Criminal damage,2,NaN,None


In [13]:
# Define the specific suburb name you want to filter by
specific_suburb = 'Ivanhoe'  # Replace 'Melbourne' with your desired suburb name

# Filter the merged GeoDataFrame for the specific suburb
filtered_gdf = merged_gdf[merged_gdf['Suburb/Town Name'] == specific_suburb]

# Display the filtered GeoDataFrame
filtered_gdf.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,sa2_name,geometry
1273,2024,March,Banyule,3079,Ivanhoe,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,4,Ivanhoe,"POLYGON ((145.02852 -37.76136, 145.02856 -37.7..."
1274,2024,March,Banyule,3079,Ivanhoe,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,14,Ivanhoe,"POLYGON ((145.02852 -37.76136, 145.02856 -37.7..."
1275,2024,March,Banyule,3079,Ivanhoe,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",4,Ivanhoe,"POLYGON ((145.02852 -37.76136, 145.02856 -37.7..."
1276,2024,March,Banyule,3079,Ivanhoe,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,11,Ivanhoe,"POLYGON ((145.02852 -37.76136, 145.02856 -37.7..."
1277,2024,March,Banyule,3079,Ivanhoe,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,8,Ivanhoe,"POLYGON ((145.02852 -37.76136, 145.02856 -37.7..."
